In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**DATA IMPORT**

In [119]:
test=pd.read_csv("/kaggle/input/ai-powered-loan-approval-prediction-model/test.csv")
train=pd.read_csv("/kaggle/input/ai-powered-loan-approval-prediction-model/train.csv")
sample=pd.read_csv("/kaggle/input/ai-powered-loan-approval-prediction-model/sample_submission.csv")

**DATA VISUAL**

In [51]:
sample.head()

,id,loan_status
0,58645,0.5
1,58646,0.5
2,58647,0.5
3,58648,0.5
4,58649,0.5


In [49]:
test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [50]:
train.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


**DATA CLEANING**

In [120]:
def cleaning_stuff(data):
    cb_person=pd.get_dummies(data["cb_person_default_on_file"],drop_first=True).astype(int).rename({"Y":"cb_person"},axis=1)
    data.drop("cb_person_default_on_file",axis=1,inplace=True)
    data=pd.concat([data,cb_person],axis=1)
    person_home_ownership=data["person_home_ownership"]
    loan_intent=data["loan_intent"]
    loan_grade=data["loan_grade"]
    person_home_ownership=pd.get_dummies(data["person_home_ownership"],drop_first=True).astype(int)
    loan_intent=pd.get_dummies(data["loan_intent"],drop_first=True).astype(int)
    loan_grade=pd.get_dummies(data["loan_grade"],drop_first=True).astype(int)
    data.drop(["loan_grade","loan_intent","person_home_ownership"],axis=1,inplace=True)
    data=pd.concat([data,person_home_ownership,loan_intent,loan_grade],axis=1)
    return data

In [121]:
test_clean=cleaning_stuff(test)
train_clean=cleaning_stuff(train)

**DATA CORELATION**

In [68]:
train_clean.corr()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status,cb_person,...,HOMEIMPROVEMENT,MEDICAL,PERSONAL,VENTURE,B,C,D,E,F,G
id,1.000000,0.008608,-0.001896,0.005376,-0.000403,-0.003454,-0.002677,0.008139,-0.008138,-0.001681,...,0.003434,0.001658,-0.010109,-0.000911,-0.003132,0.001850,-0.004138,0.000901,0.001279,0.003149
person_age,0.008608,1.000000,0.102176,0.121276,0.050378,0.009653,-0.031975,0.874260,-0.001130,0.004408,...,0.096444,0.020246,0.067335,-0.012106,-0.005142,0.001083,0.012138,0.009546,0.013077,0.002839
person_income,-0.001896,0.102176,1.000000,0.164042,0.310942,-0.057611,-0.280314,0.082727,-0.169956,-0.035514,...,0.085912,-0.062012,0.009647,0.007473,0.015667,-0.036753,-0.042052,-0.006213,-0.004014,-0.008198
person_emp_length,0.005376,0.121276,0.164042,1.000000,0.092046,-0.101910,-0.065824,0.102842,-0.100428,-0.054467,...,0.035117,-0.023727,0.021177,0.017467,-0.013930,-0.071717,-0.026486,-0.016304,-0.003289,-0.010375
loan_amnt,-0.000403,0.050378,0.310942,0.092046,1.000000,0.113582,0.647266,0.045720,0.144982,0.020271,...,0.062645,-0.034207,0.001815,0.003692,0.053154,-0.017970,0.071814,0.070362,0.032804,0.014251
loan_int_rate,-0.003454,0.009653,-0.057611,-0.101910,0.113582,1.000000,0.152201,0.007535,0.338948,0.501841,...,0.023272,0.010247,-0.010588,-0.010992,0.085884,0.449381,0.477286,0.272635,0.131935,0.075451
loan_percent_income,-0.002677,-0.031975,-0.280314,-0.065824,0.647266,0.152201,1.000000,-0.023202,0.378280,0.042979,...,-0.020208,0.023229,-0.002833,0.000417,0.036974,0.014581,0.095733,0.072407,0.039430,0.027566
cb_person_cred_hist_length,0.008139,0.874260,0.082727,0.102842,0.045720,0.007535,-0.023202,1.000000,-0.003030,0.004886,...,0.083402,0.012818,0.063577,-0.009544,-0.007549,0.007694,0.005236,0.006090,0.010322,0.001812
loan_status,-0.008138,-0.001130,-0.169956,-0.100428,0.144982,0.338948,0.378280,-0.003030,1.000000,0.186962,...,0.031063,0.049139,-0.012461,-0.064378,-0.083765,-0.009654,0.395646,0.182878,0.067644,0.045889
cb_person,-0.001681,0.004408,-0.035514,-0.054467,0.020271,0.501841,0.042979,0.004886,0.186962,1.000000,...,0.022051,-0.003625,-0.000920,-0.011662,-0.304255,0.475613,0.312687,0.131795,0.056114,0.026504


**RANDOM FOREST CLASSIFICATION**

In [100]:
from sklearn.ensemble import RandomForestClassifier

In [123]:
regressor=RandomForestClassifier()

In [125]:
x=train_clean.drop("loan_status",axis=1).values
y=train_clean[["loan_status"]].values

In [126]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=43,test_size=0.3)

In [128]:
regressor.fit(X_train,y_train)

/tmp/ipykernel_30/3585153985.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestClassifier()

In [129]:
y_pred=regressor.predict(X_test)

**METRIX ACCURACY REPORT**

In [133]:
from sklearn.metrics import classification_report,accuracy_score

In [139]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97     15655
           1       0.71      0.92      0.80      1939

    accuracy                           0.95     17594
   macro avg       0.85      0.94      0.88     17594
weighted avg       0.96      0.95      0.95     17594



In [140]:
print(accuracy_score(y_pred,y_test))

0.9488461975673526
